In [ ]:
from Mediapipe import MediapipeHands
import pandas as pd
import os

initial_dir = os.getcwd()

In [ ]:
mediapipeHands = MediapipeHands(past_data_path="/Users/andreaamaya/Trabajo/megaproyecto_model/data/data.csv")

In [ ]:
initial_sequence_id = mediapipeHands.sequence_id

In [ ]:
training_dir = f'{initial_dir}/data/videos_lensegua_3'
mediapipeHands.extract_coordinates_from_dir(training_dir)

In [ ]:
training_dir = f'{initial_dir}/data/videos_lensegua'
mediapipeHands.extract_coordinates_from_dir(training_dir)

In [ ]:
training_dir = f'{initial_dir}/data/videos_lensegua_2'
mediapipeHands.extract_coordinates_from_dir(training_dir)

In [ ]:
validation_dir = f'{initial_dir}/data/videos_val'
mediapipeHands.extract_coordinates_from_dir(validation_dir, is_val=True)

In [ ]:
validation_dir = f'{initial_dir}/data/videos_val_2'
mediapipeHands.extract_coordinates_from_dir(validation_dir, is_val=True)

In [ ]:
validation_dir = f'{initial_dir}/data/videos_val_3'
mediapipeHands.extract_coordinates_from_dir(validation_dir, is_val=True)

In [ ]:
df = mediapipeHands.get_padded_data()
validation_sequence_ids = mediapipeHands.validation_sequence_ids

In [ ]:
if (len(validation_sequence_ids) > 0):
    validation_path = f'{initial_dir}/data/validation.csv'
    validation_df = df[df['sequence_id'].isin(validation_sequence_ids)][["sequence_id", "target", "file"]]
    
    train_path = f'{initial_dir}/data/train.csv'
    train_df = df[~df['sequence_id'].isin(validation_sequence_ids)][["sequence_id", "target", "file"]]
    

    if (initial_sequence_id != 0):
        past_data = pd.read_csv(validation_path)
        final_df = pd.concat([past_data, validation_df])
        final_df.to_csv(validation_path, index=False)

        past_data = pd.read_csv(train_path)
        final_df = pd.concat([past_data, train_df])
        final_df.to_csv(train_path, index=False)
    else:
        validation_df.to_csv(validation_path, index=False)
        train_df.to_csv(train_path, index=False)

In [ ]:
data_path = f'{initial_dir}/data/data.csv'
df_final = df.drop("file", axis=1)

if (initial_sequence_id != 0):
    past_data = pd.read_csv(data_path)
    final_df = pd.concat([past_data, df_final])
    final_df.to_csv(data_path, index=False)
else:
    df_final.to_csv(data_path, index=False)

In [2]:
mediapipeHands = MediapipeHands(past_data_path="/Users/andreaamaya/Trabajo/megaproyecto_model/data/data_tests.csv")

In [3]:
initial_sequence_id = mediapipeHands.sequence_id
initial_sequence_id

7

In [4]:
extra_val_dir = f'{initial_dir}/data/test videos 2'
mediapipeHands.extract_coordinates_from_dir(extra_val_dir)

df = mediapipeHands.get_padded_data()

Hay 11 videos a extraer coordenadas


In [5]:
data_path = f'{initial_dir}/data/data_tests.csv'

if (initial_sequence_id != 0):
    past_data = pd.read_csv(data_path)
    final_df = pd.concat([past_data, df])
    final_df.to_csv(data_path, index=False)
else:
    df.to_csv(data_path, index=False)

: 